# Aim

Attempting to implement the Support Vector Kalman method, as described at http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.672.2696&rep=rep1&type=pdf.


In [1]:
from sklearn import svm
import numpy as np
from aberrations import make_vibe_params, make_vibe_data, make_noisy_data, measurement_noise, f_1
from observer import make_kfilter, vibe_fit_freq, get_psd, predict, update

In [2]:
from aberrations import time_id, f_sampling, times
W = 
N = 5
time_id /= 2 # half of it in regular KF time, half in SVK mode
steps = int(f_sampling * time_id)

In [3]:
# set up state training data
vib_freqs, vib_amps, vib_phase, vib_damping = make_vibe_params(N)
truth_states = np.array([make_vibe_data((vib_freqs[i:i+1], vib_amps[i:i+1], vib_phase[i:i+1], vib_damping[i:i+1])) 
                  for i in range(N)])
truth = make_vibe_data((vib_freqs, vib_amps, vib_phase, vib_damping))
noise = np.random.normal(0, measurement_noise, (N, times.size))
noisy_states = truth_states + noise
measurements = truth + np.sum(noise, axis=0)

In [4]:
# first, let's run the Kalman filter for half a second normally
state, A, P, Q, H, R = make_kfilter(*vibe_fit_freq(get_psd(measurements[:steps]), N=N))
states_r = np.zeros((steps, state.size))
for k in range(steps):
    state, P = update(H, P, R, state, measurements[k])
    states_r[k] = state
    state, P = predict(A, P, Q, state)

In [10]:
# now, for the next half-second, we're going to use SVR
training = states_r[steps-W-1:steps].T
svk = svm.SVR(gamma='scale', C=1.0, epsilon=0.2) # to be edited!
for training_row in training:
    svk.fit(training_row.reshape(-1,1), measurements[steps-W-1:steps])
    svk.predict()

{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, '